In [1]:
from dataset import load_dataset
from models.model_1 import ClimatePINN
from train import plot_comparison
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
import xarray as xr
from visualisation import visualize_predictions

In [2]:
temp = load_dataset(
    10, 
    train_val_split = 0.8, 
    year0=1979, 
    root_dir="./data/era_5_data",
    normalize=True)
load = DataLoader(temp['train'], batch_size=16, shuffle=True)
data = next(iter(load))

In [7]:

# # Example usage
# checkpoint_path = "checkpoints/run_1.pt"  # Update with your checkpoint path
# data_dir = "./data/era_5_data"  # Update with your data directory
# visualize_predictions(checkpoint_path, data_dir,72)